In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import requests, zipfile, io

In [2]:
postgres_password = os.environ['POSTGRES_PASSWORD']

## World Bank Data

In [3]:
#url = 'https://api.worldbank.org/v2/source/75/indicators?format=json'

r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

headers = {'User-agent': useragent,
          'From': 'jkropko@virginia.edu'}

#r = requests.get(url, headers = headers)
#myjson = json.loads(r.text)
#worldbank = pd.json_normalize(myjson)

In [4]:
#url = 'https://databank.worldbank.org/data/download/ESG_csv.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall()

In [5]:
#url = 'https://v-dem.net/media/datasets/Country_Year_V-Dem_Core_CSV_v12.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall()

labs 3/4

1. set up github and dockerfile and docker compose
2. load data
3. Pandas
4. Explain normal form
5. Implement Postgres
6. SQL

1. Create github repo called lab3and4. Choose MIT license. 

2. Wrangle

In [6]:
vdem = pd.read_csv('Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv')
vdem = vdem[['country_text_id', 'country_name','year', 'v2x_polyarchy']]
vdem = vdem.query("year >= 1960 & year <= 2021")
vdem = vdem.rename({'country_text_id':'country_code',
                    'country_name': 'country_name_vdem',
                   'v2x_polyarchy':'democracy'}, axis=1)
vdem = vdem.sort_values(['country_code', 'year'], ascending=True)
vdem

,country_code,country_name_vdem,year,democracy
5404,AFG,Afghanistan,1960,0.081
5405,AFG,Afghanistan,1961,0.083
5406,AFG,Afghanistan,1962,0.083
5407,AFG,Afghanistan,1963,0.086
5408,AFG,Afghanistan,1964,0.098
...,...,...,...,...
25976,ZZB,Zanzibar,2017,0.259
25977,ZZB,Zanzibar,2018,0.257
25978,ZZB,Zanzibar,2019,0.254
25979,ZZB,Zanzibar,2020,0.251


In [7]:
countrydata = pd.read_csv('ESGCountry.csv')

In [8]:
countrydata = countrydata[['Country Code', 'Table Name', 'Long Name', 
                           'Currency Unit', 'Region', 'Income Group']]
countrydata = countrydata.rename({'Country Code':'country_code',
                                 'Table Name':'country_name',
                                 'Long Name':'country_longname',
                                 'Currency Unit':'currency_unit',
                                 'Region':'region',
                                 'Income Group':'income_group'}, axis=1)

In [9]:
countrydata.head(1).T

,0
country_code,AFG
country_name,Afghanistan
country_longname,Islamic State of Afghanistan
currency_unit,Afghan afghani
region,South Asia
income_group,Low income


In [10]:
wb = pd.read_csv('ESGData.csv')
tokeep = [x for x in wb.columns if x.startswith('19') or x.startswith('20')]
wb = wb[['Country Code', 'Country Name', 'Indicator Code'] + tokeep]
wb = wb.drop(['2050'], axis=1)
wb = wb.rename({'Country Code':'country_code',
                'Country Name':'country_name_wb',
               'Indicator Code':'feature'}, axis=1)

In [11]:
cn = countrydata.country_name.unique()
df = pd.DataFrame({'country':cn})
df.to_csv('countries.csv')

In [12]:
wb.head(1).T

,0
country_code,ARB
country_name_wb,Arab World
feature,EG.CFT.ACCS.ZS
1960,NaN
1961,NaN
...,...
2017,86.937933
2018,87.040774
2019,87.235539
2020,87.307068


In [13]:
noncountries = ["Arab World",
"Central Europe and the Baltics",
"Caribbean small states",
"East Asia & Pacific (excluding high income)",
"Early-demographic dividend",
"East Asia & Pacific",
"Europe & Central Asia (excluding high income)",
"Europe & Central Asia",
"Euro area",
"European Union",
"Fragile and conflict affected situations",
"High income",
"Heavily indebted poor countries (HIPC)",
"IBRD only",
"IDA & IBRD total",
"IDA total",
"IDA blend",
"IDA only",
"Latin America & Caribbean (excluding high income)",
"Latin America & Caribbean",
"Least developed countries: UN classification",
"Low income",
"Lower middle income",
"Low & middle income",
"Late-demographic dividend",
"Middle East & North Africa",
"Middle income",
"Middle East & North Africa (excluding high income)",
"North America",
"OECD members",
"Other small states",
"Pre-demographic dividend",
"Pacific island small states",
"Post-demographic dividend",
"Sub-Saharan Africa (excluding high income)",
"Sub-Saharan Africa",
"Small states",
"East Asia & Pacific (IDA & IBRD)",
"Europe & Central Asia (IDA & IBRD)",
"Latin America & Caribbean (IDA & IBRD)",
"Middle East & North Africa (IDA & IBRD)",
"South Asia",
"South Asia (IDA & IBRD)",
"Sub-Saharan Africa (IDA & IBRD)",
"Upper middle income"]

In [14]:
noncountry_codes = countrydata.query("country_name in @noncountries")['country_code']

In [15]:
wb = wb.query("country_code not in @noncountry_codes")
countrydata = countrydata.query("country_code not in @noncountry_codes")

In [16]:
wb

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
2211,WLD,World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,59.315789,60.417242,61.796860,63.068478,64.299537,65.773618,67.147032,68.457171,69.681839,NaN
2212,WLD,World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.740710,85.240416,85.648321,86.619520,87.696899,88.594052,89.566532,90.011785,90.521569,NaN
2213,WLD,World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.999282,1.774908,1.515980,0.817549,0.714896,0.916155,1.239542,1.079196,0.791238,NaN
2214,WLD,World,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059747,0.061098,0.068719,0.070765,0.072687,0.069076,0.048353,0.049777,0.058882,NaN
2215,WLD,World,AG.LND.AGRI.ZS,NaN,34.55308,34.618111,34.702027,34.775809,34.871252,34.935279,...,36.629367,36.558281,36.550972,36.360611,36.336393,36.616665,36.498736,36.511307,36.458841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16008,ZWE,Zimbabwe,IC.LGL.CRED.XQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,NaN,NaN
16009,ZWE,Zimbabwe,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214746,27.214746,27.214746
16010,ZWE,Zimbabwe,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.153000,4.982000,4.770000,4.778000,4.788000,4.785000,4.796000,4.833000,5.351000,5.174000
16011,ZWE,Zimbabwe,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
wb.feature.unique()

array(['EG.CFT.ACCS.ZS', 'EG.ELC.ACCS.ZS', 'NY.ADJ.DRES.GN.ZS',
       'NY.ADJ.DFOR.GN.ZS', 'AG.LND.AGRI.ZS', 'NV.AGR.TOTL.ZS',
       'ER.H2O.FWTL.ZS', 'SI.SPR.PCAP.ZG', 'SH.DTH.COMM.ZS',
       'SL.TLF.0714.ZS', 'EN.ATM.CO2E.PC', 'CC.EST', 'EN.CLC.CDDY.XD',
       'EN.CLC.MDAT.ZS', 'IC.BUS.EASE.XQ', 'EG.ELC.COAL.ZS',
       'EG.IMP.CONS.ZS', 'EG.EGY.PRIM.PP.KD', 'EG.USE.PCAP.KG.OE',
       'SP.DYN.TFRT.IN', 'AG.PRD.FOOD.XD', 'AG.LND.FRST.ZS',
       'EG.USE.COMM.FO.ZS', 'NY.GDP.MKTP.KD.ZG', 'EN.CLC.GHGR.MT.CE',
       'SI.POV.GINI', 'GE.EST', 'SE.XPD.TOTL.GB.ZS', 'EN.CLC.HEAT.XD',
       'SH.MED.BEDS.ZS', 'SI.DST.FRST.20', 'IT.NET.USER.ZS',
       'SL.TLF.ACTI.ZS', 'SP.DYN.LE00.IN', 'SE.ADT.LITR.ZS',
       'EN.MAM.THRD.NO', 'EN.CLC.PRCP.XD', 'EN.CLC.SPEI.XD',
       'EN.ATM.METH.PC', 'SH.DYN.MORT', 'SM.POP.NETM', 'EN.ATM.NOXE.PC',
       'IP.PAT.RESD', 'SH.H2O.SMDW.ZS', 'SH.STA.SMSS.ZS',
       'EN.ATM.PM25.MC.M3', 'PV.EST', 'SP.POP.65UP.TO.ZS', 'EN.POP.DNST',
       'SI.POV.NAHC', 

In [18]:
variables = pd.read_csv('ESGSeries.csv')
variables = variables[['Series Code', 'Indicator Name']].drop_duplicates()
variables

,Series Code,Indicator Name
0,AG.LND.AGRI.ZS,Agricultural land (% of land area)
1,AG.LND.FRST.ZS,Forest area (% of land area)
2,AG.PRD.FOOD.XD,Food production index (2014-2016 = 100)
3,CC.EST,Control of Corruption: Estimate
4,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...
...,...,...
62,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)"
63,SP.DYN.TFRT.IN,"Fertility rate, total (births per woman)"
64,SP.POP.65UP.TO.ZS,Population ages 65 and above (% of total popul...
65,SP.UWT.TFRT,Unmet need for contraception (% of married wom...


In [19]:
mydict = {}
for i in range(0,67):
    x = variables.iloc[i,1]
    x = x.lower()
    x = x.split("(")[0].strip()
    x = x.split(":")[0].strip()
    x = x.split(",")[0].strip()
    x = x.split("/")[0].strip()
    x = x.replace(' ', '_')
    d = {f'{variables.iloc[i,0]}':f'{x}'}
    mydict.update(d)

In [20]:
x = 'Hello World (% of world)'
x = x.lower()
x = x.split("(")[0].strip()
x = x.split(":")[0].strip()
x = x.split(",")[0].strip()
x = x.replace(' ', '_')
x

'hello_world'

In [21]:
mydict

{'AG.LND.AGRI.ZS': 'agricultural_land',
 'AG.LND.FRST.ZS': 'forest_area',
 'AG.PRD.FOOD.XD': 'food_production_index',
 'CC.EST': 'control_of_corruption',
 'EG.CFT.ACCS.ZS': 'access_to_clean_fuels_and_technologies_for_cooking',
 'EG.EGY.PRIM.PP.KD': 'energy_intensity_level_of_primary_energy',
 'EG.ELC.ACCS.ZS': 'access_to_electricity',
 'EG.ELC.COAL.ZS': 'electricity_production_from_coal_sources',
 'EG.ELC.RNEW.ZS': 'renewable_electricity_output',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption',
 'EG.IMP.CONS.ZS': 'energy_imports',
 'EG.USE.COMM.FO.ZS': 'fossil_fuel_energy_consumption',
 'EG.USE.PCAP.KG.OE': 'energy_use',
 'EN.ATM.CO2E.PC': 'co2_emissions',
 'EN.ATM.METH.PC': 'methane_emissions',
 'EN.ATM.NOXE.PC': 'nitrous_oxide_emissions',
 'EN.ATM.PM25.MC.M3': 'pm2.5_air_pollution',
 'EN.CLC.CDDY.XD': 'cooling_degree_days',
 'EN.CLC.GHGR.MT.CE': 'ghg_net_emissions',
 'EN.CLC.HEAT.XD': 'heat_index_35',
 'EN.CLC.MDAT.ZS': 'droughts',
 'EN.CLC.PRCP.XD': 'maximum_5-day_rainfall',
 'EN.

In [22]:
replace_map = {'AG.LND.AGRI.ZS': 'agricultural_land',
 'AG.LND.FRST.ZS': 'forest_area',
 'AG.PRD.FOOD.XD': 'food_production_index',
 'CC.EST': 'control_of_corruption',
 'EG.CFT.ACCS.ZS': 'access_to_clean_fuels_and_technologies_for_cooking',
 'EG.EGY.PRIM.PP.KD': 'energy_intensity_level_of_primary_energy',
 'EG.ELC.ACCS.ZS': 'access_to_electricity',
 'EG.ELC.COAL.ZS': 'electricity_production_from_coal_sources',
 'EG.ELC.RNEW.ZS': 'renewable_electricity_output',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption',
 'EG.IMP.CONS.ZS': 'energy_imports',
 'EG.USE.COMM.FO.ZS': 'fossil_fuel_energy_consumption',
 'EG.USE.PCAP.KG.OE': 'energy_use',
 'EN.ATM.CO2E.PC': 'co2_emissions',
 'EN.ATM.METH.PC': 'methane_emissions',
 'EN.ATM.NOXE.PC': 'nitrous_oxide_emissions',
 'EN.ATM.PM25.MC.M3': 'pm2.5_air_pollution',
 'EN.CLC.CDDY.XD': 'cooling_degree_days',
 'EN.CLC.GHGR.MT.CE': 'ghg_net_emissions',
 'EN.CLC.HEAT.XD': 'heat_index_35',
 'EN.CLC.MDAT.ZS': 'droughts',
 'EN.CLC.PRCP.XD': 'maximum_5-day_rainfall',
 'EN.CLC.SPEI.XD': 'mean_drought_index',
 'EN.MAM.THRD.NO': 'mammal_species',
 'EN.POP.DNST': 'population_density',
 'ER.H2O.FWTL.ZS': 'annual_freshwater_withdrawals',
 'ER.PTD.TOTL.ZS': 'terrestrial_and_marine_protected_areas',
 'GB.XPD.RSDV.GD.ZS': 'research_and_development_expenditure',
 'GE.EST': 'government_effectiveness',
 'IC.BUS.EASE.XQ': 'ease_of_doing_business_rank',
 'IC.LGL.CRED.XQ': 'strength_of_legal_rights_index',
 'IP.JRN.ARTC.SC': 'scientific_and_technical_journal_articles',
 'IP.PAT.RESD': 'patent_applications',
 'IT.NET.USER.ZS': 'individuals_using_the_internet',
 'NV.AGR.TOTL.ZS': 'agriculture',
 'NY.ADJ.DFOR.GN.ZS': 'net_forest_depletion',
 'NY.ADJ.DRES.GN.ZS': 'natural_resources_depletion',
 'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
 'PV.EST': 'political_stability_and_absence_of_violence',
 'RL.EST': 'rule_of_law',
 'RQ.EST': 'regulatory_quality',
 'SE.ADT.LITR.ZS': 'literacy_rate',
 'SE.ENR.PRSC.FM.ZS': 'gross_school_enrollment',
 'SE.PRM.ENRR': 'primary_school_enrollment',
 'SE.XPD.TOTL.GB.ZS': 'government_expenditure_on_education',
 'SG.GEN.PARL.ZS': 'proportion_of_seats_held_by_women_in_national_parliaments',
 'SH.DTH.COMM.ZS': 'cause_of_death',
 'SH.DYN.MORT': 'mortality_rate',
 'SH.H2O.SMDW.ZS': 'people_using_safely_managed_drinking_water_services',
 'SH.MED.BEDS.ZS': 'hospital_beds',
 'SH.STA.OWAD.ZS': 'prevalence_of_overweight',
 'SH.STA.SMSS.ZS': 'people_using_safely_managed_sanitation_services',
 'SI.DST.FRST.20': 'income_share_held_by_lowest_20%',
 'SI.POV.GINI': 'gini_index',
 'SI.POV.NAHC': 'poverty_headcount_ratio_at_national_poverty_lines',
 'SI.SPR.PCAP.ZG': 'annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
 'SL.TLF.0714.ZS': 'children_in_employment',
 'SL.TLF.ACTI.ZS': 'labor_force_participation_rate',
 'SL.TLF.CACT.FM.ZS': 'ratio_of_female_to_male_labor_force_participation_rate',
 'SL.UEM.TOTL.ZS': 'unemployment',
 'SM.POP.NETM': 'net_migration',
 'SN.ITK.DEFC.ZS': 'prevalence_of_undernourishment',
 'SP.DYN.LE00.IN': 'life_expectancy_at_birth',
 'SP.DYN.TFRT.IN': 'fertility_rate',
 'SP.POP.65UP.TO.ZS': 'population_ages_65_and_above',
 'SP.UWT.TFRT': 'unmet_need_for_contraception',
 'VA.EST': 'voice_and_accountability'}

wb['feature'] = wb['feature'].replace(replace_map)

In [23]:
wb = pd.melt(wb, id_vars = ['country_code', 'country_name_wb', 'feature'], value_vars = [str(i) for i in range(1960,2022)],
            var_name = 'year')
wb

,country_code,country_name_wb,feature,year,value
0,WLD,World,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN
1,WLD,World,access_to_electricity,1960,NaN
2,WLD,World,natural_resources_depletion,1960,NaN
3,WLD,World,net_forest_depletion,1960,NaN
4,WLD,World,agricultural_land,1960,NaN
...,...,...,...,...,...
805871,ZWE,Zimbabwe,strength_of_legal_rights_index,2021,NaN
805872,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,2021,27.214746
805873,ZWE,Zimbabwe,unemployment,2021,5.174000
805874,ZWE,Zimbabwe,unmet_need_for_contraception,2021,NaN


In [24]:
wb = wb.pivot_table(index = ['country_code', 'country_name_wb', 'year'], 
                    columns = 'feature', 
                    values = 'value').reset_index()
wb

feature,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,cause_of_death,...,renewable_electricity_output,renewable_energy_consumption,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,unemployment,unmet_need_for_contraception,voice_and_accountability
0,AFG,Afghanistan,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1961,NaN,NaN,57.801696,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1962,NaN,NaN,57.893688,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1963,NaN,NaN,57.970348,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1964,NaN,NaN,58.066940,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12017,ZWE,Zimbabwe,2017,29.8,44.178635,41.876696,8.340969,27.234910,-3.5,NaN,...,NaN,82.46,NaN,-1.385256,340.45,5.0,27.214585,4.785,NaN,-1.195904
12018,ZWE,Zimbabwe,2018,29.9,45.572647,41.876696,10.520955,30.761664,NaN,NaN,...,NaN,80.23,NaN,-1.278995,359.33,5.0,27.214585,4.796,NaN,-1.140975
12019,ZWE,Zimbabwe,2019,30.1,46.781475,41.876696,10.143657,NaN,NaN,47.647301,...,NaN,81.50,NaN,-1.292071,NaN,6.0,27.214746,4.833,NaN,-1.164705
12020,ZWE,Zimbabwe,2020,30.4,52.747669,41.876696,7.607101,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.307897,NaN,NaN,27.214746,5.351,NaN,-1.113716


In [25]:
wb.year = wb.year.astype('int')

In [26]:
country_series = pd.merge(wb, vdem,
                         on = ['country_code', 'year'],
                         how = 'outer',
                         validate = 'one_to_one',
                         indicator = 'matched')

In [27]:
country_series.matched.value_counts()

both          9976
left_only     2046
right_only     395
Name: matched, dtype: int64

In [28]:
country_series.query("matched == 'left_only'").groupby(['country_code', 'country_name_wb']).agg({'year':[min, max]})

year      
                                              min   max
country_code country_name_wb                           
AND          Andorra                         1961  2021
ARE          United Arab Emirates            1960  1970
ARM          Armenia                         1960  1989
ATG          Antigua and Barbuda             1960  2021
AZE          Azerbaijan                      1960  1989
BGD          Bangladesh                      1960  1970
BHS          Bahamas, The                    1960  2021
BIH          Bosnia and Herzegovina          1960  1991
BLR          Belarus                         1960  1989
BLZ          Belize                          1960  2021
BRN          Brunei Darussalam               1960  2021
CMR          Cameroon                        1960  1960
DMA          Dominica                        1960  2021
EST          Estonia                         1960  1989
FSM          Micronesia, Fed. Sts.           1960  2021
GEO          Georgia                         1960  1989
GRD          Grenada                         1960  2021
HRV          Croatia                         1960  1990
KAZ          Kazakhstan                      1960  1989
KGZ          Kyrgyz Republic                 1960  1989
KIR          Kiribati                        1960  2021
KNA          St. Kitts and Nevis             1960  2021
LCA          St. Lucia                       1960  2021
LIE          Liechtenstein                   1961  2021
LTU          Lithuania                       1960  1989
LVA          Latvia                          1960  1989
MCO          Monaco                          1961  2021
MDA          Moldova                         1960  1989
MHL          Marshall Islands                1960  2021
MKD          North Macedonia                 1960  1990
MNE          Montenegro                      1960  1997
NRU          Nauru                           1961  2021
PLW          Palau                           1961  2021
SMR          San Marino                      1961  2021
SSD          South Sudan                     1960  2010
SVK          Slovak Republic                 1960  1992
SVN          Slovenia                        1960  1988
TJK          Tajikistan                      1960  1989
TKM          Turkmenistan                    1960  1989
TON          Tonga                           1960  2021
TUV          Tuvalu                          1960  2021
UKR          Ukraine                         1960  1989
UZB          Uzbekistan                      1960  1989
VCT          St. Vincent and the Grenadines  1960  2021
WLD          World                           1960  2021
WSM          Samoa                           1960  2021

In [29]:
country_series.query("matched == 'right_only'")[['country_code', 'country_name_vdem']].drop_duplicates()

,country_code,country_name_vdem
12022,DDR,German Democratic Republic
12053,HKG,Hong Kong
12115,PSE,Palestine/West Bank
12170,PSG,Palestine/Gaza
12192,SML,Somaliland
12223,TWN,Taiwan
12285,VDR,Republic of Vietnam
12301,XKX,Kosovo
12324,YMD,South Yemen
12355,ZZB,Zanzibar


In [30]:
country_series = pd.merge(wb, vdem,
                         on = ['country_code', 'year'],
                         how = 'inner')

## World

In [38]:
world = wb.query("country_code == 'WLD'")

In [39]:
world = world.drop(['country_code', 'country_name_wb'], axis=1)
world.columns = ['world_' + c for c in world.columns]
world = world.rename({'world_year':'year'}, axis=1)

In [40]:
world

,year,world_access_to_clean_fuels_and_technologies_for_cooking,world_access_to_electricity,world_agricultural_land,world_agriculture,world_annual_freshwater_withdrawals,world_annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income,world_cause_of_death,world_children_in_employment,world_co2_emissions,...,world_renewable_electricity_output,world_renewable_energy_consumption,world_research_and_development_expenditure,world_rule_of_law,world_scientific_and_technical_journal_articles,world_strength_of_legal_rights_index,world_terrestrial_and_marine_protected_areas,world_unemployment,world_unmet_need_for_contraception,world_voice_and_accountability
11650,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11651,1961,NaN,NaN,34.553080,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11652,1962,NaN,NaN,34.618111,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11653,1963,NaN,NaN,34.702027,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11654,1964,NaN,NaN,34.775809,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11707,2017,65.773618,88.594052,36.616665,4.139864,9.241538,NaN,NaN,NaN,4.457093,...,NaN,NaN,2.133745,NaN,2.464402e+06,5.347368,14.339432,5.557650,NaN,NaN
11708,2018,67.147032,89.566532,36.498736,3.941813,9.250457,NaN,NaN,NaN,4.510145,...,NaN,NaN,2.196333,NaN,2.554319e+06,5.507853,14.339452,5.389680,NaN,NaN
11709,2019,68.457171,90.011785,36.511307,4.007295,NaN,NaN,18.40885,NaN,4.469660,...,NaN,NaN,2.331913,NaN,NaN,5.685864,NaN,5.356915,NaN,NaN
11710,2020,69.681839,90.521569,36.458841,4.385748,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.625844,NaN,NaN,NaN,NaN,6.573234,NaN,NaN
